## Coursera capstone

This notebook is a Coursera Capstone project as a final assignment in IBM Data Science Professional Certificate

In [1]:
import pandas as pd
import numpy as np

# !pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# !conda install -c conda-forge folium=0.5.0 --yes
import folium

Get data from wikipedia

In [2]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(URL, header=0)

In [3]:
df = df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Dropping all rows which do not have Borough assigned. 

In [4]:
df = df[df.Borough != 'Not assigned']

Checking if there are Postal codes which are reoccuring and should have neighbourhoods merged.

In [5]:
len(df['Postal Code'].unique()) == len(df)

True

Checking if there are any values of 'Not assigned' in Neighbourhood column/

In [6]:
df[df.Neighbourhood == 'Not assigned']

,Postal Code,Borough,Neighbourhood


There are not but if there would be, we could run renaming.

In [7]:
for row in df.iterrows():
    if row[1].Neighbourhood == 'Not assigned':
        row[1].Neighbourhood = row[1].Borough

In [8]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df.shape

(103, 3)

In [13]:
geo_coords = pd.read_csv('./Geospatial_Coordinates.csv')

In [14]:
df_toronto = geo_coords.merge(df, on='Postal Code')

In [15]:
df_toronto.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [16]:
address = 'Toronto, CA'
df_toronto
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
#get only boroughs which have 'Toronto' in the name
df_toronto_center = df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [19]:
df_toronto_center.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M4E,43.676357,-79.293031,East Toronto,The Beaches
1,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
2,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
3,M4M,43.659526,-79.340923,East Toronto,Studio District
4,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [20]:
#show boroughs in the center of Toronto on the map
map_toronto_center = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_toronto_center['Latitude'], df_toronto_center['Longitude'], df_toronto_center['Borough'], df_toronto_center['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=False,
        parse_html=False).add_to(map_toronto_center)
    
map_toronto_center


In [88]:
#Defining Foursquare request values
CLIENT_ID = 'VM5NAACTYTTFSDBMLO0DOAUHBYVYSKD4XSEXPT504EES04ZQ'
CLIENT_SECRET = 'PO4FPTWBNZHW5AS134Z3SFO5NLPWEDJGTO1XY2SXCQ4J4AHP'
VERSION = '20210125' 
LIMIT = 100

In [29]:
df_toronto_center.loc[13]

Postal Code                            M5A
Latitude                           43.6543
Longitude                         -79.3606
Borough                   Downtown Toronto
Neighbourhood    Regent Park, Harbourfront
Name: 13, dtype: object

In [31]:
first_neigh_name = df_toronto_center.loc[13, 'Neighbourhood']
first_neigh_borough = df_toronto_center.loc[13, 'Borough']
first_neigh_lat = df_toronto_center.loc[13, 'Longitude']
first_neigh_lng = df_toronto_center.loc[13, 'Latitude']
first_neigh_pc = df_toronto_center.loc[13, 'Postal Code']
first_neigh_name

'Regent Park, Harbourfront'

In [32]:
#exploring Regent Park and Harbourfront - top 100 venues

In [60]:
radius = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    20210125, 
    first_neigh_lng, 
    first_neigh_lat, 
    radius, 
    LIMIT)

In [61]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6032832e37228e39e8d2e8ff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.6560599018, 'lng': -79.35815270005945},
   'sw': {'lat': 43.6524598982, 'lng': -79.36311909994056}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',


In [62]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Creating dataframe from the Foursquare results

In [80]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)

In [81]:
nearby_venues = nearby_venues.loc[:, ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [84]:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

In [86]:
nearby_venues.head()

,name,categories,address,lat,lng
0,Roselle Desserts,Bakery,362 King St E,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,368 King St E,43.653559,-79.361809
2,Body Blitz Spa East,Spa,497 King Street East,43.654735,-79.359874
3,Morning Glory Cafe,Breakfast Spot,457 King St. E,43.653947,-79.361149
4,The Extension Room,Gym / Fitness Center,30 Eastern Ave,43.653313,-79.359725


In [87]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


## Exploring all the neighborhoods in Central Toronto

In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print(name)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood_latitude', 
                  'neighborhood_longitude', 
                  'venue',
                  'latitude', 
                  'longitude', 
                  'category']
    
    return(nearby_venues)

In [110]:
central_toronto_venues = getNearbyVenues(df_toronto_center['Neighbourhood'], df_toronto_center['Latitude'], df_toronto_center['Longitude'])


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [114]:
central_toronto_venues.head()

,neighborhood,neighborhood_latitude,neighborhood_longitude,venue,latitude,longitude,category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [116]:
f'Found {central_toronto_venues.shape[0]} venues'

'Found 1600 venues'

In [117]:
central_toronto_venues.groupby('neighborhood').count()

,neighborhood_latitude,neighborhood_longitude,venue,latitude,longitude,category
neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


In [119]:
print('There are {} unique categories.'.format(len(central_toronto_venues['category'].unique())))

There are 233 unique categories.


## Analyzing each neighborhood

In [124]:
toronto_ohe = pd.get_dummies(central_toronto_venues[['category']], prefix="", prefix_sep="")

toronto_ohe['neighborhood'] = central_toronto_venues['neighborhood'] 

# move neighborhood column to the first column
ordered_columns = [toronto_ohe.columns[-1]] + list(toronto_ohe.columns[:-1])
toronto_ohe = toronto_ohe[ordered_columns]

toronto_ohe.head()

,neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
toronto_grouped = toronto_ohe.groupby('neighborhood').mean().reset_index()
toronto_grouped.head()

,neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.016393


#### Top 5 venues categories for each neighborhood

In [129]:
num_top_venues = 5

for hood in toronto_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.07
2          Bakery  0.05
3        Beer Bar  0.04
4  Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1         Breakfast Spot  0.08
2  Performing Arts Venue  0.08
3            Coffee Shop  0.08
4                    Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.13
1          Comic Shop  0.07
2       Auto Workshop  0.07
3                Park  0.07
4          Restaurant  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0 

#### Dataframe for 10 most common venues by neigborhood

In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [189]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = toronto_grouped['neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Japanese Restaurant,Park
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Breakfast Spot,Coffee Shop,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Garden Center,Auto Workshop,Burrito Place,Fast Food Restaurant,Farmers Market,Garden,Pizza Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Thai Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio,Ramen Restaurant


## Clustering neighborhoods

In [190]:
from sklearn.cluster import KMeans

k = 10

toronto_grouped_clustering = toronto_grouped.drop('neighborhood', 1)

kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 7, 1, 8, 1, 1, 1, 5, 8, 1, 9, 1, 1, 8, 1, 1, 6, 1, 3, 1,
       1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

Dataframe with top 10 venues and the cluster column

In [191]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'cluster', kmeans.labels_)

toronto_merged = df_toronto_center

toronto_merged = toronto_merged.merge(neighborhoods_venues_sorted.set_index('neighborhood'), left_on='Neighbourhood', right_on='neighborhood')

toronto_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,43.676357,-79.293031,East Toronto,The Beaches,4,Pub,Neighborhood,Trail,Health Food Store,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
1,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Indian Restaurant,Spa,Bookstore,Brewery
2,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West",1,Fast Food Restaurant,Sandwich Place,Gym,Sushi Restaurant,Food & Drink Shop,Ice Cream Shop,Fish & Chips Shop,Italian Restaurant,Liquor Store,Movie Theater
3,M4M,43.659526,-79.340923,East Toronto,Studio District,1,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Food,Park,Neighborhood
4,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,6,Park,Swim School,Bus Line,Farmers Market,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [200]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#colors for clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lng, neigh, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['cluster']):
    label = folium.Popup(str(neigh) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyze clusters

In [193]:
toronto_merged.loc[toronto_merged['cluster'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Latitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,43.679563,0,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [194]:
toronto_merged.loc[toronto_merged['cluster'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Latitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,43.679557,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Indian Restaurant,Spa,Bookstore,Brewery
2,43.668999,1,Fast Food Restaurant,Sandwich Place,Gym,Sushi Restaurant,Food & Drink Shop,Ice Cream Shop,Fish & Chips Shop,Italian Restaurant,Liquor Store,Movie Theater
3,43.659526,1,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Food,Park,Neighborhood
6,43.715383,1,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Seafood Restaurant,Spa,Fast Food Restaurant,Sporting Goods Shop,Salon / Barbershop,Mexican Restaurant
7,43.704324,1,Pizza Place,Dessert Shop,Sandwich Place,Gym,Coffee Shop,Italian Restaurant,Thai Restaurant,Café,Sushi Restaurant,Pharmacy
9,43.686412,1,Coffee Shop,American Restaurant,Sushi Restaurant,Pub,Light Rail Station,Fried Chicken Joint,Bank,Restaurant,Bagel Shop,Supermarket
11,43.667967,1,Coffee Shop,Bakery,Italian Restaurant,Pizza Place,Restaurant,Café,Pub,Gastropub,Liquor Store,Beer Store
12,43.665860,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Park,Men's Store,Mediterranean Restaurant
13,43.654260,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Shoe Store,Brewery
14,43.657162,1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Lingerie Store,Pizza Place


In [195]:
toronto_merged.loc[toronto_merged['cluster'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Latitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,43.711695,2,Home Service,Garden,Music Venue,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


In [196]:
toronto_merged.loc[toronto_merged['cluster'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Latitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,43.689574,3,Park,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [197]:
toronto_merged.loc[toronto_merged['cluster'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Latitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.676357,4,Pub,Neighborhood,Trail,Health Food Store,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
